In [1]:
MSASL_TRAIN_PATH = "./MS-ASL/MSASL_train.json"
MSASL_VAL_PATH = "./MS-ASL/MSASL_val.json"
MSASL_TEST_PATH = "./MS-ASL/MSASL_test.json"
MSASL_CLASSES_PATH = "./MS-ASL/MSASL_classes.json"

In [2]:
import json
import pandas as pd

with open(MSASL_TRAIN_PATH) as f:
    train_data = json.load(f)

with open(MSASL_VAL_PATH) as f:
    val_data = json.load(f)

with open(MSASL_TEST_PATH) as f:
    test_data = json.load(f)
    
with open(MSASL_CLASSES_PATH) as f:
    class_data = json.load(f)

train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)
class_df = pd.DataFrame(class_data)

class_df.columns = ['label_name']
class_df['label'] = class_df.index

train_df = train_df.merge(class_df, how='left', on='label')
val_df = val_df.merge(class_df, how='left', on='label')
test_df = test_df.merge(class_df, how='left', on='label')

In [3]:
train_df

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,label_name
0,match [light-a-MATCH],match,0.000,0,0,0,83,match light-a-MATCH,830,360.0,30.000,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN,match
1,FAIL,fail,0.000,0,-1,0,74,FAIL,542,360.0,25.000,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN,fail
2,laugh,laugh,0.000,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.970,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN,laugh
3,BOOK,book,0.000,0,-1,0,66,BOOK(3),38,360.0,25.000,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN,book
4,sign-language,sign language,0.000,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.970,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN,sign language
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16049,WRITE,write,539.939,32,17,16182,16301,ASL 1 Unit 1 Vocabulary,53,360.0,29.970,543.910,https://www.youtube.com/watch?v=fNg_sJ9f8EI,write,"[0.03217703104019165, 0.3161315321922302, 1.0,...",640.0,NaN,write
16050,hot,hot,0.000,2,42,0,127,Hot,135,360.0,29.657,4.282,https://www.youtube.com/watch?v=MkXUHhsMAns,hot,"[0.00036713480949401855, 0.20649650692939758, ...",640.0,NaN,hot
16051,hi,hi,0.000,0,0,0,47,Hi,379,360.0,29.970,1.568,www.youtube.com/watch?v=rhfJGeMDMzQ,hi,"[0.013705313205718994, 0.17314371466636658, 1....",640.0,NaN,hi
16052,Learn,learn,0.000,135,-1,0,104,ASL Learn,22,360.0,30.000,3.467,https://www.youtube.com/watch?v=n-b2NMAwk28,learn,"[0.18269836902618408, 0.19255371391773224, 1.0...",480.0,NaN,learn


In [8]:
import os
import pandas as pd
import yt_dlp
import glob
import shutil

from moviepy.video.io.VideoFileClip import VideoFileClip


OUTPUT_DIR = "./msasl_clips"
NUMBER_OF_CLIPS = 50

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)


open("skipped_videos.txt", "w").close()

def download_and_clip(row, split):
    video_url = row["url"]
    start = float(row["start_time"])
    end = float(row["end_time"])
    label = row["label_name"]
    filename_base = row["file"].replace(" ", "_")

    label_dir = os.path.join(OUTPUT_DIR, split, label)
    full_video_path_no_ext = os.path.join(label_dir, f"{filename_base}_full")
    clipped_path = os.path.join(label_dir, f"{filename_base}_clip.mp4")

    if os.path.exists(clipped_path):
        return

    ydl_opts = {
        "format": "bestvideo+bestaudio/best",
        "merge_output_format": "mp4",
        "outtmpl": full_video_path_no_ext,  
        "quiet": True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
    except Exception as e:
        error_msg = str(e).lower()
        if "private video" in error_msg or "video unavailable" in error_msg:
            print(f"Skipping private/unavailable video: {video_url}")
            with open("skipped_videos.txt", "a") as logf:
                logf.write(f"{video_url}\n")
        else:
            print(f"Failed to download {video_url}: {e}")
        return

    downloaded_files = glob.glob(full_video_path_no_ext + "*")
    if not downloaded_files:
        print(f"Failed to find downloaded file for {video_url}")
        return

    downloaded_path = downloaded_files[0]

    try:
        os.makedirs(label_dir, exist_ok=True)

        with VideoFileClip(downloaded_path) as video:
            clip = video.subclip(start, end)
            clip.write_videofile(clipped_path, codec="libx264", audio=False, verbose=False, logger=None)

        os.remove(downloaded_path)

    except Exception as e:
        print(f"Failed to clip {downloaded_path}: {e}")
        return

splits = {
    "train": train_df,
    "val": val_df,
    "test": test_df
}

for split_name, df in splits.items():
    df = df[["url", "start_time", "end_time", "label", "label_name", "file"]]
    clips_to_process = df.head(NUMBER_OF_CLIPS) if NUMBER_OF_CLIPS else df
    
    for _, row in clips_to_process.iterrows():
        download_and_clip(row, split_name)

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 1202, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 2.88, 'bitrate': 1193, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1920, 1080], 'video_bitrate': 1202, 'video_fps': 30.0, 'default

ERROR: [youtube] 9FdHlMOnVjg: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Skipping private/unavailable video: www.youtube.com/watch?v=9FdHlMOnVjg
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 480], 'bitrate': 496, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 2.76, 'bitrate': 491, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Main)', 'video_siz

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 1271, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 2.62, 'bitrate': 1257, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1920, 1080], 'video_bitrate': 1271, 'video_fps': 2

ERROR: [youtube] 1AyT77LqJzQ: Video unavailable


...

Skipping private/unavailable video: www.youtube.com/watch?v=OdJuIwj6iYY


ERROR: [youtube] BBCixbI_6C4: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Skipping private/unavailable video: www.youtube.com/watch?v=BBCixbI_6C4


ERROR: [youtube] BL90-ejIQ9Y: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Skipping private/unavailable video: www.youtube.com/watch?v=BL90-ejIQ9Y
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1279, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 383.94, 'bitrate': 1285, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(

In [4]:
from IPython.display import Video

Video("./msasl_clips/train/book/BOOK(3)_full.mp4", embed=True, width=640, height=360)

In [5]:
import os
import torch
import torch.nn as nn
import random
import numpy as np

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_video
from decord import VideoReader
from decord import cpu


class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, num_frames=30):
        self.root_dir = root_dir
        self.transform = transform
        self.num_frames = num_frames
        self.classes = sorted(os.listdir(root_dir))
        self.video_paths = []

        for label in self.classes:
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                for file in os.listdir(label_dir):
                    if file.endswith(".mp4"):
                        self.video_paths.append((os.path.join(label_dir, file), label))

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path, label = self.video_paths[idx]

        vr = VideoReader(video_path, ctx=cpu(0))
        total_frames = len(vr)

        indices = np.linspace(0, total_frames - 1, self.num_frames, dtype=int)
        frames = vr.get_batch(indices).asnumpy()

        images = [Image.fromarray(frame) for frame in frames]
        if self.transform:
            images = [self.transform(img) for img in images]
        
        frames_tensor = torch.stack(images)

        label_idx = self.classes.index(label)
        return frames_tensor, label_idx


In [6]:
video_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),  
    transforms.RandomHorizontalFlip(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

In [7]:
train_dir = "./msasl_clips/train/"

train_dataset = VideoDataset(root_dir=train_dir, transform=video_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=2,
    pin_memory=False,
    persistent_workers=True
)

for frames, labels in train_loader:
    print(f"Frames shape: {frames.shape}")
    print(f"Labels: {labels}")
    break

Frames shape: torch.Size([16, 30, 3, 128, 128])
Labels: tensor([23, 18, 15, 21,  0, 26,  3, 10, 12, 11, 24,  5,  6,  4,  7, 28])


In [10]:
val_dir = "./msasl_clips/val/"

val_dataset = VideoDataset(root_dir=val_dir, transform=video_transforms)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,  
    shuffle=True,
    num_workers=2,
    persistent_workers=True
)

for frames, labels in val_loader:
    print(f"frames shape: {frames.shape}")
    print(f"labels: {labels}")
    break

frames shape: torch.Size([9, 30, 3, 128, 128])
labels: tensor([6, 6, 1, 3, 2, 0, 7, 4, 5])


In [11]:
test_dir = "./msasl_clips/test/"

test_dataset = VideoDataset(root_dir=test_dir, transform=video_transforms)

test_loader = DataLoader(
    test_dataset,
    batch_size=16,  
    shuffle=True,
    num_workers=3,  
    persistent_workers=True
)

for frames, labels in test_loader:
    print(f"frames shape: {frames.shape}")
    print(f"labels: {labels}")
    break

frames shape: torch.Size([16, 30, 3, 128, 128])
labels: tensor([ 4, 10, 21, 18,  2,  5, 23, 16,  7, 20, 22, 24,  1, 12,  8, 19])


In [14]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetForVideoClassification(nn.Module):
    def __init__(self, num_classes, num_frames=30):
        super().__init__()
        self.num_frames = num_frames
        
        base_model = models.resnet18(pretrained=True)
        base_model.fc = nn.Identity()  
        
        self.resnet = base_model
        self.fc = nn.Linear(base_model.fc.in_features if hasattr(base_model.fc, 'in_features') else 512, num_classes)

    def forward(self, x):
        batch_size, frames, c, h, w = x.shape 
        
        x = x.view(-1, c, h, w)  
        
        features = self.resnet(x)  
        features = features.view(batch_size, frames, -1) 
        features = features.mean(dim=1)  
        
        return self.fc(features)  

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(train_dataset.classes)
num_frames = 30  

model = ResNetForVideoClassification(num_classes=num_classes, num_frames=num_frames).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for frames, labels in train_loader:
        frames = frames.to(device)  
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for frames, labels in val_loader:
            frames = frames.to(device)
            labels = labels.to(device)
            outputs = model(frames)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= len(val_loader)
    val_accuracy = 100 * val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10] Train Loss: 3.5538, Train Acc: 3.23% | Val Loss: 3.7291, Val Acc: 0.00%
